In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import socket
import struct
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split


In [7]:
# Load data
df = pd.read_csv('dataset.csv')

# Convert 'date' to datetime and set as index
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# Assuming 'index' column is to be dropped
df.drop('index', axis=1, inplace=True)

# Split dataset function
def split_dataset(data):
    train, test = data[:27000], data[27000:27000+15840]
    train = np.array(np.split(train, len(train)//10))
    test = np.array(np.split(test, len(test)//10))
    return train, test

# Convert to supervised
def to_supervised(train, n_input, n_out=60):
	data = train.reshape((train.shape[0]*train.shape[1], 1))
	X, y = list(), list()
	in_start = 0
	for _ in range(len(data)):
		in_end = in_start + n_input
		out_end = in_end + n_out
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 0])
		in_start += 1
	return np.array(X), np.array(y)

# Apply split and supervised transformation
train, test = split_dataset(df)
train_x, train_y = to_supervised(train, 10, 10)

In [8]:
def create_lstm_model():
    model = Sequential([
        LSTM(200, input_shape=(10, 1), activation='relu'),  # Adjust input_shape based on your dataset
        Dense(100, activation='relu'),
        Dense(10)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    return model

In [9]:
def send_msg(sock, msg):
    msg = pickle.dumps(msg)
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recv_msg(sock):
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    return pickle.loads(recvall(sock, msglen))

def recvall(sock, n):
    data = bytearray()
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data.extend(packet)
    return data


In [10]:
def client_main(host='localhost', port=12345):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((host, port))

    # Receive initial model weights
    weights = recv_msg(s)
    
    # Load and prepare data
    # The data loading and preprocessing code goes here, already provided above
    
    # Create and compile the model
    model = create_lstm_model()
    model.set_weights(weights)  # Set model weights to received weights
    
    # Train model on client's data
    model.fit(train_x, train_y, epochs=1, batch_size=32, verbose=1)
    
    # Send updated weights back to server
    updated_weights = model.get_weights()
    send_msg(s, updated_weights)
    
    s.close()

if __name__ == "__main__":
    client_main()


Train on 26981 samples
26981/26981 [==============================] - 26s 958us/sample - loss: 142.9662 - accuracy: 0.0970
